<a href="https://colab.research.google.com/github/ggaoshen/graph_wavenet/blob/sg-experiment/CS224W_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CS224W - Final Project**

Description goes here.

# Device
You might need to use GPU for this Colab.

Please click `Runtime` and then `Change runtime type`. Then set the `hardware accelerator` to **GPU**.

# Installation

In [1]:
# Install torch geometric
import os
import torch
if 'IS_GRADESCOPE_ENV' not in os.environ:
  torch_version = str(torch.__version__)
  scatter_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
  sparse_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
  !pip install torch-scatter -f $scatter_src
  !pip install torch-sparse -f $sparse_src
  !pip install torch-geometric
  !pip install -q git+https://github.com/snap-stanford/deepsnap.git
  !pip install -U -q PyDrive

Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 47.6 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  !nvcc --version
  !python -c "import torch; print(torch.version.cuda)"

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
11.8


In [3]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  import torch
  print(torch.__version__)
  import torch_geometric
  print(torch_geometric.__version__)

2.1.0+cu118
2.4.0


In [4]:
# Import GraphWaveNet module
!git clone https://github.com/ggaoshen/graph_wavenet.git
%load graph_wavenet/src/graphwavenet.py
import sys
sys.path.append('graph_wavenet/src/')
from graphwavenet import GraphWaveNet

Cloning into 'graph_wavenet'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 86 (delta 39), reused 37 (delta 9), pack-reused 0
Receiving objects: 100% (86/86), 32.22 KiB | 10.74 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [5]:
# Install and import TGB dataset
!pip install py-tgb
from tgb.nodeproppred.dataset_pyg import PyGNodePropPredDataset

dataset_name = "tgbn-trade"
dataset = PyGNodePropPredDataset(name=dataset_name, root="datasets")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for clint: filename=clint-0.5.1-py3-none-any.whl size=34457 sha256=8f090528d1e1e76060dd19cc821cdae3fe56650b141794dafebcc02ba435d5c7
  Stored in directory: /root/.cache/pip/wheels/60/bd/a0/c20dd085251d95af656b5e3d287db7d4b4e8aec67b53a6f8dd
  Created wheel for args: filename=args-0.1.0-py3-none-any.whl size=3318 sha256=e5cfc68c1da6e3931b1ec4de1b98cca67f9f177f2de34ec5d571192de2519276
  Stored in directory: /root/.cache/pip/wheels/18/d7/bc/7b88d8405d97070a1a62712fd639ea0ad8d14b3dd74075cca6
Successfully built clint args
Will you download the dataset(s) now? (y/N)
y
Download started, this might take a while . . . 
Dataset title: tgbn-trade
Download completed 
Dataset directory is  /usr/local/lib/python3.10/dist-packages/tgb/datasets/tgbn_trade
file not processed, generating processed file
number of lines counted 4682

468246it [00:01, 254267.74it/s]
458775it [00:01, 410930.36it/s]


file processed and saved


In [6]:
# Get stats of TGB dataset
data = dataset.get_TemporalData()

num_nodes = data.num_nodes # GraphWaveNet input
num_edges = data.num_edges
num_timesteps = len(data.t.unique()) # GraphWaveNet input
print(f"{dataset_name}: {num_nodes} nodes, {num_edges} edges and {num_timesteps} timesteps")

tgbn-trade: 255 nodes, 468245 edges and 31 timesteps


In [18]:
class NodePredictor(torch.nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.lin_node = torch.nn.Linear(in_dim, in_dim)
        self.out = torch.nn.Linear(in_dim, out_dim)

    def forward(self, node_embed):
        h = self.lin_node(node_embed)
        h = h.relu()
        h = self.out(h)
        # h = F.log_softmax(h, dim=-1)
        return h


import math
from torch_geometric.nn import TransformerConv
import torch


class GraphAttentionEmbedding(torch.nn.Module):
    """
    Reference:
    - https://github.com/pyg-team/pytorch_geometric/blob/master/examples/tgn.py
    """

    def __init__(self, in_channels, out_channels, msg_dim, time_enc):
        super().__init__()
        self.time_enc = time_enc
        edge_dim = msg_dim + time_enc.out_channels
        self.conv = TransformerConv(
            in_channels, out_channels // 2, heads=2, dropout=0.1, edge_dim=edge_dim
        )

    def forward(self, x, last_update, edge_index, t, msg):
        rel_t = last_update[edge_index[0]] - t
        rel_t_enc = self.time_enc(rel_t.to(x.dtype))
        edge_attr = torch.cat([rel_t_enc, msg], dim=-1)
        return self.conv(x, edge_index, edge_attr)




In [19]:
next(iter(dataset))

TemporalData(src=0, dst=1, t=1986, msg=[1], y=1.0)

In [20]:
def train(model, memory, node_pred, train_loader, optimizer, criterion, device):
    model.train()
    memory.train()
    node_pred.train()

    total_loss = 0
    total_correct = 0
    total_examples = 0

    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()

        # Extracting source and destination nodes
        src, dst = batch.src, batch.dst

        # Forming the edge index from source and destination nodes
        edge_index = torch.stack([src, dst], dim=0)

        # Optional: if you have edge weights, create the edge_weight tensor
        # For example, using 'msg' as a proxy for edge weights (modify as needed)
        edge_weight = batch.msg

        # Update memory state (if applicable in your model)
        memory.update_state(src, dst, batch.t, batch.msg)

        # Prepare node features (x) and labels (y)
        # This depends on how your model expects node features to be represented
        # As an example, you might create a one-hot encoding of node IDs, or use a learned embedding
        x = ...  # Compute/extract node features
        y = batch.y

        # Forward pass through GraphWaveNet
        out = model(x, edge_index, edge_weight=edge_weight)

        # Forward pass through the node prediction layer
        pred = node_pred(out)

        # Compute loss - assuming a classification task
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()

        # Compute accuracy - for classification tasks
        total_correct += pred.max(dim=1)[1].eq(y).sum().item()
        total_loss += loss.item()
        total_examples += y.size(0)

    avg_loss = total_loss / len(train_loader)
    accuracy = total_correct / total_examples

    return avg_loss, accuracy


In [24]:
from functools import reduce
from operator import mul

import torch
from torch import nn
from torch.nn import functional as F

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import DenseGCNConv, GCNConv


class GraphWaveNet(nn.Module):
    r""" GraphWaveNet implementation from the
    `Graph WaveNet for Deep Spatial-Temporal Graph Modeling
    <https://arxiv.org/abs/1906.00121>`_ paper. The model takes a list of
    node embeddings across several time steps and predicts the output
    embeddings for the specified number of upcoming time steps.
    .. note::
        For examples of the GraphWaveNet layer, see
        `examples/contrib/graphwavenet/main.py
        <https://github.com/pyg-team/pytorch_geometric/blob/master/examples/
        contrib/graphwavenet/main.py>`_ .
    Args:
        num_nodes (int): Number of nodes in the input graph.
        in_channels (int): Size of each input sample.
        out_channels (int): Size of each output sample.
        out_timesteps (int): The number of time steps for which predictions
            are generated.
        dilations (list(int), optional): The set of dilations applied by each
            temporal convolution layer. For each dilation :obj:`d`, the
            temporal convolution captures information from :obj:`d` time steps
            prior. (default: :obj:`[1, 2, 1, 2, 1, 2, 1, 2]`)
        adaptive_embeddings (int, optional): The size of the embeddings used
            to calculate the adaptive matrix. (default: :obj:`10`)
        dropout (float, optional): Dropout probability. (default: :obj:`0.3`)
        residual_channels (int, optional): Number of residual channels
            (default: :obj:`32`)
        dilation_channels (int, optional): Number of dilation channels
            (default: :obj:`32`)
        skip_channels (int, optional): Number of skip layer channels
            (default: :obj:`256`)
        end_channels (int, optional): Size of the final linear layer
            (default: :obj:`512`)
    """
    def __init__(self, num_nodes, in_channels, out_channels, out_timesteps,
                 dilations=[1, 2, 1, 2, 1, 2, 1, 2], adptive_embeddings=10,
                 dropout=0.3, residual_channels=32, dilation_channels=32,
                 skip_channels=256, end_channels=512):

        super(GraphWaveNet, self).__init__()

        self.total_dilation = sum(dilations)
        self.num_dilations = len(dilations)
        self.num_nodes = num_nodes

        self.dropout = dropout

        self.e1 = nn.Parameter(torch.randn(num_nodes, adptive_embeddings),
                               requires_grad=True)
        self.e2 = nn.Parameter(torch.randn(adptive_embeddings, num_nodes),
                               requires_grad=True)

        self.input = nn.Conv2d(in_channels=in_channels,
                               out_channels=residual_channels,
                               kernel_size=(1, 1))

        self.tcn_a = nn.ModuleList()
        self.tcn_b = nn.ModuleList()
        self.gcn = nn.ModuleList()
        self.gcn_adp = nn.ModuleList()
        self.bn = nn.ModuleList()
        self.skip = nn.ModuleList()

        self.out_timesteps = out_timesteps
        self.out_channels = out_channels

        for d in dilations:
            self.tcn_a.append(
                nn.Conv2d(in_channels=residual_channels,
                          out_channels=dilation_channels, kernel_size=(1, 2),
                          dilation=d))

            self.tcn_b.append(
                nn.Conv2d(in_channels=residual_channels,
                          out_channels=dilation_channels, kernel_size=(1, 2),
                          dilation=d))

            # GCNConv is used for performing graph convolutions over the
            # normal adjacency matrix
            self.gcn.append(
                GCNConv(in_channels=dilation_channels,
                        out_channels=residual_channels))

            # Since the adaptive matrix is a softmax output, it represents a
            # dense graph
            # For fast training and inference, we use a DenseGCNConv layer
            self.gcn_adp.append(
                DenseGCNConv(in_channels=dilation_channels,
                             out_channels=residual_channels))

            self.skip.append(
                nn.Conv2d(in_channels=residual_channels,
                          out_channels=skip_channels, kernel_size=(1, 1)))
            self.bn.append(nn.BatchNorm2d(residual_channels))

        self.end1 = nn.Conv2d(in_channels=skip_channels,
                              out_channels=end_channels, kernel_size=(1, 1))
        # self.end2 = nn.Conv2d(in_channels=end_channels,
        #                       out_channels=out_channels * out_timesteps,
        #                       kernel_size=(1, 1))

    def forward(self, x, edge_index, edge_weight=None):
        r"""
        The forward pass for the GraphWaveNet
        Args:
            x (torch.Tensor): The temporal node features of shape
                :math:`(*, t_{input}, |\mathcal{V}|, F_{in})`
            edge_index (torch.Tensor): The edge indices
                of shape :math:`(2, |\mathcal{E}|)`
            edge_weight (torch.Tensor, optional): The edge weights
                shape of :math:`(|\mathcal{E}|)` (default: :obj:`None`)
        Return types:
            * **x** (*torch.Tensor*): Predicted temporal node features of shape
              :math:`(*, t_{output}, |\mathcal{V}|, F_{out})`
        """

        # Both batched and unbatched input is supported
        # In the case of a single batch, the input is broadcast
        print(x.size())
        print(edge_index.size())
        # print(edge_weight.size())
        is_batched = True
        if len(x.size()) == 3:
            is_batched = False
            x = torch.unsqueeze(x, dim=0)

        batch_size = x.size()[:-3]
        print(x.size())
        input_timesteps = x.size(-3)

        x = x.transpose(-1, -3)

        # If the dilation steps require a larger number of input time steps,
        # padding is performed
        if self.total_dilation + 1 > input_timesteps:
            x = F.pad(x, (self.total_dilation - input_timesteps + 1, 0))

        x = self.input(x)

        # The adaptive adjacency matrix is the product of 2 learnable
        # embedding matrices
        # ReLU is used toeliminate weak connections
        # SoftMax function is applied to normalize the self-adaptive adjacency
        # matrix
        adp = F.softmax(F.relu(torch.mm(self.e1, self.e2)), dim=1)

        skip_out = None

        for k in range(self.num_dilations):
            residual = x

            # TCN layer
            g1 = self.tcn_a[k](x)
            g1 = torch.tanh(g1)

            g2 = self.tcn_b[k](x)
            g2 = torch.sigmoid(g2)

            g = g1 * g2

            # The skip connection output is aggregated before the GCN layer
            skip_cur = self.skip[k](g)

            if skip_out is None:
                skip_out = skip_cur
            else:
                # Since dilation reduces the number of time steps,
                # only the required number of latest time steps are considered
                skip_out = skip_out[..., -skip_cur.size(-1):] + skip_cur

            g = g.transpose(-1, -3)

            timesteps = g.size(-3)
            g = g.reshape(reduce(mul, g.size()[:-2]), *g.size()[-2:])

            # The data for several time steps in batched into a single batch
            # This helps speed up the model by passing a single large batch to
            # the GCN
            data = self.__batch_timesteps__(g, edge_index,
                                            edge_weight).to(g.device)

            # GCN layer
            # One GCN is for the actual adjacency matrix
            # The other GCN is for the adaptive matrix
            gcn_out = self.gcn[k](data.x, data.edge_index,
                                  edge_weight=torch.flatten(data.edge_attr))
            gcn_out = gcn_out.reshape(*batch_size, timesteps, -1,
                                      self.gcn[k].out_channels)

            gcn_out_adp = self.gcn_adp[k](g, adp)
            gcn_out_adp = gcn_out_adp.reshape(*batch_size, timesteps, -1,
                                              self.gcn[k].out_channels)

            x = gcn_out + gcn_out_adp

            x = F.dropout(x, p=self.dropout)

            x = x.transpose(-3, -1)

            # The residual connection is fed to the next spatial-temporal layer
            x = x + residual[..., -x.size(-1):]
            x = self.bn[k](x)

        skip_out = skip_out[..., -1:]

        x = torch.relu(skip_out)

        # Final linear layer
        x = self.end1(x)
        # x = torch.relu(x)


        # Transforming the output to the appropriate number of channels
        # (out_timesteps * self.out_channels)
        # x = self.end2(x)

        # The output is reshaped into the expected final shape
        if is_batched:
            x = x.reshape(*batch_size, self.out_timesteps, self.out_channels,
                          self.num_nodes).transpose(-1, -2)
        else:
            x = x.reshape(self.out_timesteps, self.out_channels,
                          self.num_nodes).transpose(-1, -2)

        return x

    def __batch_timesteps__(self, x, edge_index, edge_weight=None):
        r"""
        This method batches the data for several time steps into a single batch
        """
        edge_index = edge_index.expand(x.size(0), *edge_index.shape)

        if edge_weight is not None:
            edge_weight = edge_weight.expand(x.size(0), *edge_weight.shape)

        dataset = [
            Data(x=_x, edge_index=e_i, edge_attr=e_w)
            for _x, e_i, e_w in zip(x, edge_index, edge_weight)
        ]
        loader = DataLoader(dataset=dataset, batch_size=x.size(0))

        return next(iter(loader))

In [25]:
# train_data.size()

# data.msg.size(-1)

# print(data)
next(iter(data))

# GraphWaveNet(
#         in_channels=memory_dim,
#         out_channels=embedding_dim,
#         num_nodes=num_nodes,
#         out_timesteps=num_timesteps,
#         # msg_dim=data.msg.size(-1),
#         # time_enc=memory.time_enc,
#     )

TemporalData(src=[1], dst=[1], t=[1], msg=[1, 1], y=[1])

In [27]:
# TGB Example - not working yet

import timeit
import argparse
from tqdm import tqdm
import torch
import matplotlib.pyplot as plt

from torch_geometric.loader import TemporalDataLoader
from torch_geometric.nn import TGNMemory
from torch_geometric.nn.models.tgn import (
    IdentityMessage,
    LastAggregator,
    LastNeighborLoader,
)

# from modules.decoder import NodePredictor
# from modules.emb_module import GraphAttentionEmbedding
from tgb.nodeproppred.dataset_pyg import PyGNodePropPredDataset
from tgb.nodeproppred.evaluate import Evaluator
from tgb.utils.utils import set_random_seed
from tgb.utils.stats import plot_curve

# parser = argparse.ArgumentParser(description='parsing command line arguments as hyperparameters')
# parser.add_argument('-s', '--seed', type=int, default=1,
#                     help='random seed to use')
# parser.parse_args("")
# args = parser.parse_args("")
# setting random seed
# seed = int(args.seed) #1,2,3,4,5
seed = 1
torch.manual_seed(seed)
set_random_seed(seed)

# hyperparameters
lr = 0.0001
epochs = 50

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_mask = dataset.train_mask
val_mask = dataset.val_mask
test_mask = dataset.test_mask

eval_metric = dataset.eval_metric
num_classes = dataset.num_classes
data = data.to(device)

evaluator = Evaluator(name=dataset_name)


train_data = data[train_mask]
val_data = data[val_mask]
test_data = data[test_mask]

# Ensure to only sample actual destination nodes as negatives.
min_dst_idx, max_dst_idx = int(data.dst.min()), int(data.dst.max())

batch_size = 200

train_loader = TemporalDataLoader(train_data, batch_size=batch_size)
val_loader = TemporalDataLoader(val_data, batch_size=batch_size)
test_loader = TemporalDataLoader(test_data, batch_size=batch_size)

neighbor_loader = LastNeighborLoader(data.num_nodes, size=10, device=device)

memory_dim = time_dim = embedding_dim = 100

memory = TGNMemory(
    data.num_nodes,
    data.msg.size(-1),
    memory_dim,
    time_dim,
    message_module=IdentityMessage(data.msg.size(-1), memory_dim, time_dim),
    aggregator_module=LastAggregator(),
).to(device)

print("GWN input dimensions", memory_dim, embedding_dim, num_nodes, num_timesteps)
gnn = (
    GraphWaveNet(
        in_channels=memory_dim,
        out_channels=embedding_dim,
        num_nodes=num_nodes,
        out_timesteps=num_timesteps,
        # msg_dim=data.msg.size(-1),
        # time_enc=memory.time_enc,
    )
    .to(device)
    .float()
)

# gnn = (
#     GraphAttentionEmbedding(
#         in_channels=memory_dim,
#         out_channels=embedding_dim,
#         msg_dim=data.msg.size(-1),
#         time_enc=memory.time_enc,
#     )
#     .to(device)
#     .float()
# )


node_pred = NodePredictor(in_dim=embedding_dim, out_dim=num_classes).to(device)

optimizer = torch.optim.Adam(
    set(memory.parameters()) | set(gnn.parameters()) | set(node_pred.parameters()),
    lr=lr,
)

criterion = torch.nn.CrossEntropyLoss()
# Helper vector to map global node indices to local ones.
assoc = torch.empty(data.num_nodes, dtype=torch.long, device=device)


def plot_curve(scores, out_name):
    plt.plot(scores, color="#e34a33")
    plt.ylabel("score")
    plt.savefig(out_name + ".pdf")
    plt.close()


def process_edges(src, dst, t, msg):
    if src.nelement() > 0:
        # msg = msg.to(torch.float32)
        memory.update_state(src, dst, t, msg)
        neighbor_loader.insert(src, dst)


def train():
    memory.train()
    gnn.train()
    node_pred.train()

    memory.reset_state()  # Start with a fresh memory.
    neighbor_loader.reset_state()  # Start with an empty graph.

    total_loss = 0
    label_t = dataset.get_label_time()  # check when does the first label start
    print(label_t)
    num_label_ts = 0
    total_score = 0

    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        src, dst, t, msg = batch.src, batch.dst, batch.t, batch.msg
        # print(src.size(), dst.size(), t.size(), msg.size())
        # print(torch.stack([src, dst], dim=0).size())

        query_t = batch.t[-1]
        # check if this batch moves to the next day
        if query_t > label_t:
            # find the node labels from the past day
            label_tuple = dataset.get_node_label(query_t)
            label_ts, label_srcs, labels = (
                label_tuple[0],
                label_tuple[1],
                label_tuple[2],
            )
            label_t = dataset.get_label_time()
            label_srcs = label_srcs.to(device)

            # Process all edges that are still in the past day
            previous_day_mask = batch.t < label_t
            process_edges(
                src[previous_day_mask],
                dst[previous_day_mask],
                t[previous_day_mask],
                msg[previous_day_mask],
            )
            # Reset edges to be the edges from tomorrow so they can be used later
            src, dst, t, msg = (
                src[~previous_day_mask],
                dst[~previous_day_mask],
                t[~previous_day_mask],
                msg[~previous_day_mask],
            )

            """
            modified for node property prediction
            1. sample neighbors from the neighbor loader for all nodes to be predicted
            2. extract memory from the sampled neighbors and the nodes
            3. run gnn with the extracted memory embeddings and the corresponding time and message
            """
            n_id = label_srcs
            n_id_neighbors, mem_edge_index, e_id = neighbor_loader(n_id)
            assoc[n_id_neighbors] = torch.arange(n_id_neighbors.size(0), device=device)

            z, last_update = memory(n_id_neighbors)
            # print("Z AND Last update")
            # print(z.size())
            # print(last_update.size())

            z = gnn(
                # z,
                # last_update,
                # mem_edge_index,
                # data.t[e_id].to(device),
                # data.msg[e_id].to(device),
                x=z.reshape(z.size()[0], -1, z.size()[1]),  # The node features
                edge_index=mem_edge_index,  # The edge indices
                edge_weight=data.msg[e_id].to(device)  # Optional edge weights, if your GraphWaveNet uses them
            )
            z = z[assoc[n_id]]

            # loss and metric computation
            pred = node_pred(z)

            loss = criterion(pred, labels.to(device))
            np_pred = pred.cpu().detach().numpy()
            np_true = labels.cpu().detach().numpy()

            input_dict = {
                "y_true": np_true,
                "y_pred": np_pred,
                "eval_metric": [eval_metric],
            }
            result_dict = evaluator.eval(input_dict)
            score = result_dict[eval_metric]
            total_score += score
            num_label_ts += 1

            loss.backward()
            optimizer.step()
            total_loss += float(loss)

        # Update memory and neighbor loader with ground-truth state.
        process_edges(src, dst, t, msg)
        memory.detach()

    metric_dict = {
        "ce": total_loss / num_label_ts,
    }
    metric_dict[eval_metric] = total_score / num_label_ts
    return metric_dict


@torch.no_grad()
def test(loader):
    memory.eval()
    gnn.eval()
    node_pred.eval()
    total_score = 0
    label_t = dataset.get_label_time()  # check when does the first label start
    num_label_ts = 0

    for batch in loader:
        batch = batch.to(device)
        src, dst, t, msg = batch.src, batch.dst, batch.t, batch.msg

        query_t = batch.t[-1]
        if query_t > label_t:
            label_tuple = dataset.get_node_label(query_t)
            if label_tuple is None:
                break
            label_ts, label_srcs, labels = (
                label_tuple[0],
                label_tuple[1],
                label_tuple[2],
            )
            label_t = dataset.get_label_time()
            label_srcs = label_srcs.to(device)

            # Process all edges that are still in the past day
            previous_day_mask = batch.t < label_t
            process_edges(
                src[previous_day_mask],
                dst[previous_day_mask],
                t[previous_day_mask],
                msg[previous_day_mask],
            )
            # Reset edges to be the edges from tomorrow so they can be used later
            src, dst, t, msg = (
                src[~previous_day_mask],
                dst[~previous_day_mask],
                t[~previous_day_mask],
                msg[~previous_day_mask],
            )

            """
            modified for node property prediction
            1. sample neighbors from the neighbor loader for all nodes to be predicted
            2. extract memory from the sampled neighbors and the nodes
            3. run gnn with the extracted memory embeddings and the corresponding time and message
            """
            n_id = label_srcs
            n_id_neighbors, mem_edge_index, e_id = neighbor_loader(n_id)
            assoc[n_id_neighbors] = torch.arange(n_id_neighbors.size(0), device=device)

            z, last_update = memory(n_id_neighbors) # what is last_update used for??
            z = gnn(
                z,
                last_update,
                mem_edge_index,
                data.t[e_id].to(device),
                data.msg[e_id].to(device),
            )
            # z = gnn(
            #     x=z,  # The node features
            #     edge_index=mem_edge_index,  # The edge indices
            #     edge_weight=data.msg[e_id].to(device)  # Optional edge weights, if your GraphWaveNet uses them
            # )
            z = z[assoc[n_id]]

            # loss and metric computation
            pred = node_pred(z)
            np_pred = pred.cpu().detach().numpy()
            np_true = labels.cpu().detach().numpy()

            input_dict = {
                "y_true": np_true,
                "y_pred": np_pred,
                "eval_metric": [eval_metric],
            }
            result_dict = evaluator.eval(input_dict)
            score = result_dict[eval_metric]
            total_score += score
            num_label_ts += 1

        process_edges(src, dst, t, msg)

    metric_dict = {}
    metric_dict[eval_metric] = total_score / num_label_ts
    return metric_dict


train_curve = []
val_curve = []
test_curve = []
max_val_score = 0  #find the best test score based on validation score
best_test_idx = 0
for epoch in range(1, epochs + 1):
    start_time = timeit.default_timer()
    train_dict = train()
    print("------------------------------------")
    print(f"training Epoch: {epoch:02d}")
    print(train_dict)
    train_curve.append(train_dict[eval_metric])
    print("Training takes--- %s seconds ---" % (timeit.default_timer() - start_time))

    start_time = timeit.default_timer()
    val_dict = test(val_loader)
    print(val_dict)
    val_curve.append(val_dict[eval_metric])
    if (val_dict[eval_metric] > max_val_score):
        max_val_score = val_dict[eval_metric]
        best_test_idx = epoch - 1
    print("Validation takes--- %s seconds ---" % (timeit.default_timer() - start_time))

    start_time = timeit.default_timer()
    test_dict = test(test_loader)
    print(test_dict)
    test_curve.append(test_dict[eval_metric])
    print("Test takes--- %s seconds ---" % (timeit.default_timer() - start_time))
    print("------------------------------------")
    dataset.reset_label_time()


# code for plotting
plot_curve(train_curve, "train_curve")
plot_curve(val_curve, "val_curve")
plot_curve(test_curve, "test_curve")

max_test_score = test_curve[best_test_idx]
print("------------------------------------")
print("------------------------------------")
print ("best val score: ", max_val_score)
print ("best validation epoch   : ", best_test_idx + 1)
print ("best test score: ", max_test_score)

GWN input dimensions 100 100 255 31
1989
torch.Size([204, 1, 100])
torch.Size([2, 2030])
torch.Size([1, 204, 1, 100])


RuntimeError: ignored